In [503]:
# // i-th iteration
# f(x_2, x_1, y_2, y_1) {
#     x = x_1 + x_2 // Fibonacci
#     y = y_1 * y_2 // Multiplicative Fibonacci
#     t = x * y     // Fibonacci x (Multiplicative Fibonacci)
#     (x, y, t)
# }

k = GF(101)

M1 = Matrix(k, [
    [1,1,0,0,0,0,0,0],
    [0,0,0,0,1,0,0,0],
    [0,0,1,0,0,0,0,0],
    [0,0,0,0,0,0,0,0]
])

M2 = Matrix(k, [
    [0,0,0,0,0,0,0,1],
    [0,0,0,1,0,0,0,0],
    [0,0,0,0,0,1,0,0],
    [0,0,0,0,0,0,0,0]
])

M3 = Matrix(k, [
    [0,0,1,0,0,0,0,0],
    [0,0,0,0,0,1,0,0],
    [0,0,0,0,0,0,1,0],
    [0,0,0,0,0,0,0,0]
])

m = M1.nrows()
s1 = log(m,2)
n = M1.ncols()
s2 = log(n,2)

# First iteration
# TODO break it up into x, w, 1
x = [0,1,1,2,3,6,6]
w = []
z1 = vector(k, x + w + [1])

# Second iteration
# z2 = vector(k, [1,1,2,3,6,18,36,1])

$$
1 \cdot (M_1 \cdot z \circ M_2 \cdot z) + (-1) \cdot (M_3 \cdot z) = \mathbf{0}
$$

In [504]:
# CCS relation
1 * (M1*z1.column()).elementwise_product(M2*z1.column())-(M3*z1.column())

[0]
[0]
[0]
[0]

In [505]:
R = PolynomialRing(k, 10, "x1, x2, y1, y2, y3, x11, x22, y11, y22, y33")
x1, x2, y1, y2, y3, x11, x22, y11, y22, y33 = R.gens()

$$
\widetilde{eq}(X_1, X_2) := \prod_{j=1}^l ((1-X_{1_j})(1-X_{2_j}) + X_{1_j} X_{2_j})
$$

In [506]:
import numpy

eqx = numpy.prod([(1 - xi)*(1 - xii) + xi*xii for (xi, xii) in [(x1, x11), (x2, x22)]])
    
eqy = numpy.prod([(1 - yi)*(1 - yii) + yi*yii for (yi, yii) in [(y1, y11), (y2, y22), (y3, y33)]])

eqx(x1=1, x2=1, x11=1, x22=0)

0

In [507]:
row = x1 + 2*x2
col = y1 + 2*y2 + 4*y3

$$
\widetilde{M_i}(X, Y) := \sum_{x \in \{0, 1 \}^{s}} \sum_{y \in \{0, 1 \}^{s'}} M_i(x, y) \cdot \widetilde{eq}(x, X) \cdot \widetilde{eq}(y, Y)
$$

In [508]:
def Mi_linear(Mi):
    return sum([
            sum([
                sum([
                    sum([
                        sum([
                            Mi[Integer(row(x1=x1, x2=x2))][Integer(col(y1=y1,y2=y2,y3=y3))] * eqx(x1=x1,x2=x2,x11=x11,x22=x22) * eqy(y1=y1, y11=y11, y2=y2, y22=y22, y3=y3, y33=y33)
                        for y3 in [0,1]])
                    for y2 in [0,1]])
                for y1 in [0,1]])
            for x2 in [0,1]])
         for x1 in [0,1]])

Mi_linear(M1)(x11=0,x22=0,y11=1,y22=0,y33=0)

1

$$
\widetilde{z}((Y_1, Y_2, Y_3)) := \sum_{y_1 \in \{0, 1 \}}\sum_{y_2 \in \{0, 1 \}}\sum_{y_3 \in \{0, 1 \}} z((y_1, y_2, y_3)) \cdot \widetilde{eq}((y_1, y_2, y_3), (Y_1, Y_2, Y_3))
$$

In [509]:
def z_linear(zi):
    return sum([
                sum([
                    sum([
                        zi[Integer(col(y1=y1,y2=y2,y3=y3))]* eqy(y1=y1, y11=y11, y2=y2, y22=y22, y3=y3, y33=y33)
                    for y3 in [0,1]])
                for y2 in [0,1]])
            for y1 in [0,1]])

z_linear(z1)(y11=1,y22=0,y33=0)

1

$$
\sum_{y \in \{0, 1 \}^3} \widetilde{M_1}((X_1, X_2), y) \cdot \widetilde{z}(y)
$$

In [510]:
def Mi_z_prod(Mi, zi):
        return sum([
                sum([
                    sum([
                        Mi_linear(Mi)(y11=y1,y22=y2,y33=y3) * z_linear(zi)(y11=y1,y22=y2,y33=y3)
                    for y3 in [0,1]])
                for y2 in [0,1]])
            for y1 in [0,1]])

$$
\begin{align*}
G((X_1, X_2)) := & 1 \cdot (\sum_{y \in \{0, 1 \}^3} \widetilde{M_1}((X_1, X_2), y) \cdot \widetilde{z}(y) \cdot \sum_{y \in \{0, 1 \}^{3}} \widetilde{M_1}((X_1, X_2), y) \cdot \widetilde{z}(y)) \\  + & (-1) \cdot (\sum_{y \in \{0, 1 \}^{3}} \widetilde{M_3}((X_1, X_2), y) \cdot \widetilde{z}(y)) = \mathbf{0}
\end{align*}
$$

In [511]:
G = Mi_z_prod(M1, z1) * Mi_z_prod(M2, z1) - Mi_z_prod(M3, z1)
G

21*x11^2*x22^2 - 17*x11^2*x22 - 15*x11*x22^2 + 2*x11^2 + 11*x11*x22 - 2*x11

$$
h((X_1, X_2)) = \sum_{x_1 \in \{0,1\}}\sum_{x_2 \in \{0,1\}}  G((x_1, x_2)) \cdot \widetilde{eq}((X_1, X_2), (x_1, x_2)) = 0
$$

In [512]:
h = sum([
        sum([
            G(x11=x11, x22=x22) * eqx(x11=x11, x22=x22)
            for x22 in [0,1]])
     for x11 in [0,1]])

h

0

In [513]:
# Verifier samples a random challenge beta:=(beta_1, beta_2)
beta1 = k.random_element()
beta2 = k.random_element()

(beta1, beta2)

(31, 36)

$$
Q(X_1, X_2) := G((X_1, X_2)) \cdot \widetilde{eq}((\beta_1, \beta_2), (X_1, X_2))
$$

In [514]:
Q = G * eqx(x1=beta1,x2=beta2)
Q

-50*x11^3*x22^3 + 11*x11^3*x22^2 - 9*x11^2*x22^3 + 12*x11^3*x22 - 40*x11^2*x22^2 + 34*x11*x22^3 - 28*x11^3 - 20*x11^2*x22 + 8*x11*x22^2 + 7*x11^2 - 47*x11*x22 + 21*x11

## Outer Sum-Check

$$
\sum_{x_1 \in \{0,1\}}\sum_{x_2 \in \{0,1\}} Q(x_1, x_2) = 0
$$

In [515]:
# Sumcheck protocol 1

# Prover claims that h = 0
sum([ 
    sum([ Q(x11=x1, x22=x2) for x2 in [0,1]])
for x1 in [0,1]]) == 0

True

$$
Q_1(X_1) := \sum_{x_2 \in \{0,1\}} Q(X_1, x_2)
$$

In [516]:
# Round 1
# Prover sends s1 claiming to be Q1
Q1 = sum([ Q(x22=x2) for x2 in [0,1]])
s1 = Q1

s1

18*x11^3 + 46*x11^2 + 37*x11

In [517]:
# Verifier checks 
s1(x11=0) + s1(x11=1) == 0

True

In [518]:
# Verifier computes a random element to check that s1 is actually Q1
r1 = k.random_element()
r1

28

$$
Q_2(X_2) := Q(r_1, X_2)
$$

In [519]:
# Round 2 (final)
# Prover sends s2 claiming to be Q2
Q2 = Q(x11=r1)
s2 = Q2

s2

24*x22^3 - 47*x22^2 - 12*x22 + 46

In [520]:
# Verifier checks 
s2(x22=0) + s2(x22=1) == s1(x11=r1)

True

$$
Q((r_1, r_2)) = G((r_1, r_2)) \cdot \widetilde{eq}((\beta_1, \beta_2), (r_1, r_2))
$$

In [521]:
# Verifier computes a random element to check that s2 is actually Q(r1, r2)
r2 = k.random_element()
r2

# Verifier wants to compute Q himself, so he does so with another sumcheck

83

## Inner Sum-Check

$$
\begin{align}
T_i &= \sum_{y \in \{0, 1 \}^3} \widetilde{M_i}((r_1, r_2), y) \cdot \widetilde{z}(y) \\
&= \sum_{y_1 \in \{0, 1 \}} \sum_{y_2 \in \{0, 1 \}} \sum_{y_3 \in \{0, 1 \}} \widetilde{M_i}((r_1, r_2), (y_1, y_2, y_3)) \cdot \widetilde{z}((y_1, y_2, y_3)) \\
\end{align}
$$

#### T1 Sum-Check

In [522]:
# Sumcheck protocol 2

# Prover claims that for r1, r2, the value T1 is equal to f(r1, r2)
def f(Mi, zi):
    return sum([
        sum([
            sum([
                Mi_linear(Mi)(x11=r1, x22= r2, y11=y1,y22=y2,y33=y3) * z_linear(zi)(y11=y1,y22=y2,y33=y3)
            for y3 in [0,1]])
        for y2 in [0,1]])
    for y1 in [0,1]])

T1 = f(M1, z1)
T1

# We will do the same for T2 and T3

-47

$$
f_1(Y_1) := \sum_{y_2 \in \{0,1\}}\sum_{y_3 \in \{0,1\}} \widetilde{M_i}((r_1, r_2), (Y_1, y_2, y_3)) \cdot \widetilde{z}((Y_1, y_2, y_3))
$$

In [523]:
# Round 1
# Prover sends q1 claiming to be f1
f1 = sum([
        sum([
            Mi_linear(M1)(x11=r1, x22= r2, y22=y2,y33=y3) * z_linear(z1)(y22=y2,y33=y3)
        for y3 in [0,1]])
    for y2 in [0,1]])
q1 = f1

# Note that q1 is only 3 coefficients
q1

39*y11^2 - 8*y11 - 39

In [524]:
# Verifier checks 
q1(y11=0) + q1(y11=1) == T1

True

In [525]:
# Verifier computes a random element to check that q1 is actually f2
r11 = k.random_element()
r11

85

$$
f_2(Y_2) := \sum_{y_3 \in \{0,1\}}\widetilde{M_i}((r_1, r_2), (r'_1, Y_2, y_3)) \cdot \widetilde{z}((r'_1, y_2, y_3))
$$

In [526]:
# Round 2
# Prover sends s2 claiming to be f2
f2 = sum([
        Mi_linear(M1)(x11=r1, x22= r2, y11=r11, y33=y3) * z_linear(z1)(y11=r11,y33=y3)
    for y3 in [0,1]])
q2 = f2

# Note that s2 is only 3 coefficients
q2

-46*y22^2 - 34*y22 - 24

In [527]:
# Verifier checks 
q2(y22=0) + q2(y22=1) == q1(y11=r11)

True

In [528]:
# Verifier computes a random element to check that q2 is actually f2
r22 = k.random_element()
r22

34

$$
f_3(Y_3) := \widetilde{M_i}((r_1, r_2), (r'_1, r'_2, Y_3)) \cdot \widetilde{z}((r'_1, r'_2, Y_3))
$$

In [529]:
# Round 2
# Prover sends q3 claiming to be f3
f3 = Mi_linear(M1)(x11=r1, x22= r2, y11=r11, y22=r22) * z_linear(z1)(y11=r11,y22=r22)
q3 = f3

# Note that q3 is only 3 coefficients
q3

13*y33^2 - 3*y33 - 14

In [530]:
# Verifier checks 
q3(y33=0) + q3(y33=1) == q2(y22=r22)

True

In [531]:
# Verifier computes a random element to check that q3 is actually f3
r33 = k.random_element()
r33

57

$$
f_3(r'_3) == \widetilde{M_i}((r_1, r_2), (r'_1, r'_2, r'_3)) \cdot \widetilde{z}((r'_1, r'_2, r'_3))
$$

In [532]:
# Verifier computes f(r11, r22, r33) 
c1 = Mi_linear(M1)(x11=r1, x22= r2, y11=r11,y22=r22,y33=r33) * z_linear(z1)(y11=r11,y22=r22,y33=r33)

c1 == q3(y33=r33)

True

#### T2 Sum-Check (same as T1)

In [533]:
# Repeat same sucheck for M2
T2 = f(M2, z1)
T2

33

In [534]:
# Round 1
# Prover sends q1 claiming to be f1
f1 = sum([
        sum([
            Mi_linear(M2)(x11=r1, x22= r2, y22=y2,y33=y3) * z_linear(z1)(y22=y2,y33=y3)
        for y3 in [0,1]])
    for y2 in [0,1]])
q1 = f1

# Note that q1 is only 3 coefficients
q1

10*y11^2 + 23*y11

In [535]:
# Verifier checks 
q1(y11=0) + q1(y11=1) == T2

True

In [536]:
# Verifier computes a random element to check that s1 is actually f2
r11 = k.random_element()
r11

41

In [537]:
# Round 2
# Prover sends q2 claiming to be f2
f2 = sum([
        Mi_linear(M2)(x11=r1, x22= r2, y11=r11, y33=y3) * z_linear(z1)(y11=r11,y33=y3)
    for y3 in [0,1]])
q2 = f2

# Note that q2 is only 3 coefficients
q2

-28*y22^2 - 31*y22 + 18

In [538]:
# Verifier checks 
q2(y22=0) + q2(y22=1) == q1(y11=r11)

True

In [539]:
# Verifier computes a random element to check that s2 is actually f2
r22 = k.random_element()
r22

30

In [540]:
# Round 2
# Prover sends q3 claiming to be f3
f3 = Mi_linear(M2)(x11=r1, x22= r2, y11=r11, y22=r22) * z_linear(z1)(y11=r11,y22=r22)
q3 = f3

# Note that s3 is only 3 coefficients
q3

44*y33^2 - 26*y33 - 36

In [541]:
# Verifier checks 
q3(y33=0) + q3(y33=1) == q2(y22=r22)

True

In [542]:
# Verifier computes a random element to check that s3 is actually f3
r33 = k.random_element()
r33

11

In [543]:
# Verifier computes
c2 = Mi_linear(M2)(x11=r1, x22= r2, y11=r11,y22=r22,y33=r33) * z_linear(z1)(y11=r11,y22=r22,y33=r33)

c2 == q3(y33=r33)

True

#### T3 Sum-Check (same as T1, T2)

In [544]:
# Repeat same sucheck for M3
T3 = f(M3, z1)
T3

40

In [545]:
# Round 1
# Prover sends q1 claiming to be f1
f1 = sum([
        sum([
            Mi_linear(M3)(x11=r1, x22= r2, y22=y2,y33=y3) * z_linear(z1)(y22=y2,y33=y3)
        for y3 in [0,1]])
    for y2 in [0,1]])
q1 = f1

# Note that q1 is only 3 coefficients
q1

-6*y11^2 - 13*y11 - 21

In [546]:
# Verifier checks 
q1(y11=0) + q1(y11=1) == T3

True

In [547]:
# Verifier computes a random element to check that q1 is actually f2
r11 = k.random_element()
r11

60

In [548]:
# Round 2
# Prover sends q2 claiming to be f2
f2 = sum([
        Mi_linear(M3)(x11=r1, x22= r2, y11=r11, y33=y3) * z_linear(z1)(y11=r11,y33=y3)
    for y3 in [0,1]])
q2 = f2

# Note that q2 is only 3 coefficients
q2

34*y22^2 + 38*y22 + 25

In [549]:
# Verifier checks 
q2(y22=0) + q2(y22=1) == q1(y11=r11)

True

In [550]:
# Verifier computes a random element to check that q2 is actually f2
r22 = k.random_element()
r22

51

In [551]:
# Round 2
# Prover sends q3 claiming to be f3
f3 = Mi_linear(M3)(x11=r1, x22= r2, y11=r11, y22=r22) * z_linear(z1)(y11=r11,y22=r22)
q3 = f3

# Note that q3 is only 3 coefficients
q3

y33^2 + 28*y33 + 37

In [552]:
# Verifier checks 
q3(y33=0) + q3(y33=1) == q2(y22=r22)

True

In [553]:
# Verifier computes a random element to check that s3 is actually f3
r33 = k.random_element()
r33

48

In [554]:
# Verifier computes
c3 = Mi_linear(M3)(x11=r1, x22= r2, y11=r11,y22=r22,y33=r33) * z_linear(z1)(y11=r11,y22=r22,y33=r33)

c3 == q3(y33=r33)

True

## Final check

$$
\begin{align*}
G((r_1, r_2)) &= 1 \cdot (\sum_{y \in \{0, 1 \}^3} \widetilde{M_1}((r_1, r_2),  y) \cdot \widetilde{z}(y) \cdot \sum_{y \in \{0, 1 \}^{3}} \widetilde{M_1}((r_1, r_2), y) \cdot \widetilde{z}(y)) \\
& + (-1) \cdot (\sum_{y \in \{0, 1 \}^{3}} \widetilde{M_3}((r_1, r_2), y) \cdot \widetilde{z}(y)) \\
&= T_1 \cdot T_2 - T_3
\end{align*}
$$

In [555]:
G(x11=r1, x22=r2) == (T1 * T2 - T3)

True

In [556]:
Q(x11=r1, x22=r2)

0

In [557]:
(T1 * T2 - T3)

25

In [558]:
eqx(x11=r1, x22=r2, x1=beta1, x2=beta2)

0

$$
\begin{align*}
s_2(r_2) = Q((r_1, r_2)) &= G((r_1, r_2)) \cdot \widetilde{eq}((\beta_1, \beta_2), (r_1, r_2)) \\
&= (T_1 \cdot T_2 - T_3) \cdot \widetilde{eq}((\beta_1, \beta_2), (r_1, r_2))
\end{align*}
$$

In [559]:
# verifier can now compute that Q(r1, r2) = (T1 * T2 - T3) * eq((r1, r2), (beta1, beta2))

Q(x11=r1, x22=r2) == (T1 * T2 - T3) * eqx(x11=r1, x22=r2, x1=beta1, x2=beta2)

True